Connect Robot, Thymio Suite must be open and thymio must be connected to pc!

In [1]:
import math
import time
import ast
import openai
import tdmclient.notebook
import tdmclient.tools

In [2]:
await tdmclient.notebook.start()

Thymio Program

In [33]:
%%run_python

state = 0


@onevent
def forward(distance):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 100 * distance
    motor_left_target = 200
    motor_right_target = 200
    emit("motion", 1)

@onevent
def backward(distance):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 100 * distance
    motor_left_target = -200
    motor_right_target = -200
    emit("motion", 1)

@onevent
def rotateleft(angle):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 35 * angle
    motor_left_target = -50
    motor_right_target = 100
    emit("motion", 1)

@onevent
def rotateright(angle):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 35 * angle
    motor_left_target = 100
    motor_right_target = -50
    emit("motion", 1)

@onevent
def prox():
    global motor_left_target, motor_right_target, prox_horizontal, state
    if state == 0:
        for i in range(5):
            if prox_horizontal[i] > 2000:
                state = 1
                timer_period[0] = 1000
                motor_left_target = 0
                motor_right_target = 0
                emit("problem", 0)
    if state == 3:
        motor_right_target = (prox_horizontal[0] + prox_horizontal[1]) // 20
        motor_left_target = (prox_horizontal[3] + prox_horizontal[4]) // 20


@onevent
def turn(right_speed, left_speed, timer):
    global motor_left_target, motor_right_target, timer_period
    motor_left_target = left_speed
    motor_right_target = right_speed
    timer_period[0] = timer * 1000
    emit("motion", 1)


@onevent
def follow(timer):
    global state
    state = 3
    timer_period[0] = timer

@onevent
def stopfollow(_):
    global state
    state = 0
    timer_period[0] = 5

@onevent
def timer0():
    global motor_left_target, motor_right_target
    if state == 0:
        timer_period[0] = 0
        motor_left_target = 0
        motor_right_target = 0
        emit("done", 0)
    if state == 1:
        state = 0
    



In [51]:
import math

class Done(Exception):
    pass

class Problem(Exception):
    pass

def parse_input(task_str):
    actions = ast.literal_eval(task_str)
    return actions

def calculate_turn(radius, angle):
    wheel_distance = 9.4
    velocity = 5.8
    
    radian_angle = math.radians(angle)  # Convert angle from degrees to radians
    arc_length = radius * abs(radian_angle)  # Calculate the arc length the robot needs to travel

    # Calculate the target velocities for each wheel
    if angle == 0:
        motor_left_target = motor_right_target = velocity
    else:
        motor_right_target = velocity * (1 + wheel_distance / (2 * radius)) if angle > 0 else velocity * (1 - wheel_distance / (2 * radius))
        motor_left_target = velocity * (1 - wheel_distance / (2 * radius)) if angle > 0 else velocity * (1 + wheel_distance / (2 * radius))

    return int(motor_left_target * 100/2.685), int(motor_right_target * 100/2.685), int(arc_length / velocity)


def pop_and_send(event_list) -> bool:
    if len(event_list) > 0:
        event = event_list.pop(0)
        event_name = event[0]
        match event_name:
            case "turn":
                right_speed, left_speed, timer = calculate_turn(event[1], event[2])
                print(right_speed, left_speed, timer)
                send_event("turn", right_speed, left_speed, timer)
            case _:
                send_event(event[0], *event[1:])
        return True
    else:
        return False

def execute_actions(actions): 
    event_list = actions.copy()

    def on_event_data(node, event_name):
        data = get_event_data(event_name)
        print(data)
        if event_name == "done":
            clear_event_data(event_name)
            success = pop_and_send(event_list)
            if not success:
                raise Done
        elif event_name == "problem":
            clear_event_data(event_name)
            raise Problem
        else:
            clear_event_data(event_name)

    pop_and_send(event_list)
    try:
        tdmclient.notebook.process_events(all_nodes=True, on_event_data=on_event_data)
    except Done:
        print("Robot task completed successfully")
        return "done", None
    except Problem:
        print("Robot encounted a problem")
        return "problem", event_list


@tdmclient.notebook.sync_var
def main():
    quit = False
    tasks = [parse_input(input("Please input a list of actions for the robot to perform"))]
    while not quit:
        response, left_over_events = execute_actions(tasks.pop())
        match response:
            case "done":
                if len(tasks) == 0:
                    quit = True
            case "problem":
                if left_over_events is not None:
                    tasks.append(left_over_events)
                tasks.append(parse_input(input("Robot encoutered a problem (*Insert short description*).\nPlease provice extra guidance.")))
            

Unchecked MatchAs()


In [53]:
main()

ZeroDivisionError: float division by zero

ChatGPT integration